In [66]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import cv2
import sys
import warnings
import os

In [67]:
images_format=('.png','.jpeg','.jpg','.tiff','.tif')
print(images_format)

('.png', '.jpeg', '.jpg', '.tiff', '.tif')


In [68]:
#function to check if a file is in correct format
def is_correct_file(im_format,file):
    if file.endswith(im_format) :
        return 0
    else :
        return -1

In [69]:
#function for gaussian blur filter
def gaussian_blur_filter(image, size):
    blurred_image = cv2.GaussianBlur(image, (size, size), 0,0)
    return blurred_image

#function for laplacian filter
def laplacian_filter(image):
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGBA2GRAY)
    
    laplacian = cv2.Laplacian(gray_image, cv2.CV_8U)
    
    enhanced_image = cv2.addWeighted(gray_image, 1.5, laplacian, -0.5, 0)
    
    enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGBA)
    
    return enhanced_image

#function for color boosting
def color_boosting(image, channel):
    
    if len(image.shape)!=3 or image.shape[2]!=4 or channel<0 or channel>=3 :
        return None
    boosted_image = np.array(image, copy=True).astype(np.uint32)

    boosted_image[:,:,channel] = np.clip(boosted_image[:,:,channel]+50, 0, 255)
    
    boosted_image = np.array(boosted_image, copy=True).astype(np.uint8)

    return boosted_image

#function for sharpening filter
def sharpening_filter(image) :
    sharp = np.array([[0, -1, 0],
                     [-1, 5, -1],
                     [0, -1, 0]])
    
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    sharp_image = cv2.filter2D(image,-1,sharp)
    sharp_image = cv2.cvtColor(sharp_image, cv2.COLOR_RGB2RGBA)
    return sharp_image

#function for sepia filter
def sepia_filter(image):
    sepia = np.array([[0.272, 0.534, 0.131],
                     [0.349, 0.686, 0.168],
                     [0.393, 0.769, 0.189]])
    
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    sepia_image = cv2.transform(image, sepia)
    sepia_image = cv2.cvtColor(sepia_image, cv2.COLOR_RGB2RGBA)

    return sepia_image

#function for box blur filter
def box_blur_filter(image) :
    box_blur = np.array([[1/9, 1/9, 1/9],
                        [1/9, 1/9, 1/9],
                        [1/9, 1/9, 1/9]])
  
    bluring_image = cv2.filter2D(image,-1,box_blur)
    return bluring_image

#function for face detection and put meme(mask or hat)
def face_detection(image,meme_path,hat):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    detected_faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    meme_asset = cv2.imread(meme_path, cv2.IMREAD_UNCHANGED)
    meme_asset = cv2.cvtColor(meme_asset, cv2.COLOR_BGRA2RGBA)
    
    if len(detected_faces)==0 :
        return None
    
    x, y, w, h = detected_faces[0]
    if hat is not True :
        y= y + h // 20
        x= x + w // 40
        resized = cv2.resize(meme_asset, (w, h))
        mask = resized[:, :, 3] / 255.0
        rgb = resized[:, :, :3]
        overlay = (1.0 - mask[:, :, np.newaxis]) * image[y:y+h, x:x+w, :3] + mask[:, :, np.newaxis] * rgb
        overlay = overlay.astype(np.uint8)
        image[y:y+h, x:x+w, :3] = overlay
    else :
        if y-50 <= 0:
            return None
        w_hat = w
        h_hat = int(h*0.7)
        y_hat = max(y - int(h * 0.4),0)
        x_hat = x + w // 40
        
        hat_image_resized = cv2.resize(meme_asset, (w_hat, h_hat))
        region_of_interest = image[y_hat:y_hat+h_hat, x_hat:x_hat+w_hat]
        
        if hat_image_resized.shape[:2] != region_of_interest.shape[:2]:
            resized_hat = cv2.resize(hat_image_resized, (w_hat, h_hat))
        else:
            resized_hat = hat_image_resized

        masked_hat = resized_hat[:, :, :3] * (resized_hat[:, :, 3:] / 255.0)
        masked_image = region_of_interest[:, :, :3] * (1.0 - resized_hat[:, :, 3:] / 255.0)
        combined = (masked_hat + masked_image).astype(np.uint8)

        image[y_hat:y_hat+h_hat, x_hat:x_hat+w_hat, :3] = combined
        
    return image

#function for eyes detection and put meme(eye mask or glasses)
def eyes_detection(image,meme_path):
    eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    detected_eyes = eyes_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    meme_asset = cv2.imread(meme_path, cv2.IMREAD_UNCHANGED)
    meme_asset = cv2.cvtColor(meme_asset, cv2.COLOR_BGRA2RGBA)
    meme_alpha = meme_asset[:, :, 3] / 255.0
    
    detected_eyes = sorted(detected_eyes, key=lambda x: x[0])
   
    if len(detected_eyes) < 2 :
        return None
    
    x_1, y_1, w_1, h_1 = detected_eyes[0]
    x_2, y_2, w_2, h_2 = detected_eyes[1]
    
    meme_width = x_2 + int(2.2*w_2) - x_1
    meme_height = int(2.2*max(h_1, h_2))
    
    resized_meme = cv2.resize(meme_asset[:, :, :3], (meme_width, meme_height))
    resized_meme_alpha = cv2.resize(meme_alpha, (meme_width, meme_height))
    
    x_offset = x_1 - int(meme_width  // 7.3)
    y_offset = y_1 - int(meme_height // 3.5)
    
    reshaped_meme_alpha = np.expand_dims(resized_meme_alpha, axis=2)

    overlay = (image[y_offset:y_offset + meme_height, x_offset:x_offset + meme_width] * (1 - reshaped_meme_alpha)).astype(np.uint8)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_RGBA2RGB)
    image[y_offset:y_offset + meme_height, x_offset:x_offset + meme_width, :3] = (resized_meme * reshaped_meme_alpha + overlay)
    
    return image

In [70]:
while True :
    image_name = input("Enter the image name from folder project_images or Press 'Exit' to exit: ")
    if image_name=="Exit" or image_name=="exit" :
        break
    path_for_image = os.path.join('project_images', image_name)
    result=is_correct_file(images_format,path_for_image)
    if result==-1 :
        print("\n\""+path_for_image+"\""+" is NOT acceptable format! Please use a jpg/png/tiff image\n")
        error=-1
    else :
        if not os.path.exists(path_for_image) :
            print("\n\""+path_for_image+"\""+" does not exist\n")
            error=-1
        else :
            print("\n\""+path_for_image+"\""+" is OK\n")
            image_cv2 = cv2.imread(path_for_image, cv2.IMREAD_UNCHANGED)
            image_cv2 = cv2.cvtColor(image_cv2,cv2.COLOR_BGRA2RGBA)
            image_from_filters=image_cv2.copy()
            error=1
    while error!=-1 :
        print("\nMenu for filters and other options below\n0: Exit from the filters\n1: Go to memes\n2: Gaussian Blur Filter\n3: Laplacian Filter\n4: Color Boosting(for coloring images)\n5: Sharpening Filter\n6: Sepia Filter\n7: Box Blur Filter\n")
        code_for_filter = int(input("Enter the option from menu: "))
        
        if code_for_filter==0 :
            break
        elif code_for_filter==1 : 
            while True :
                meme=image_from_filters.copy()
                print("\nMenu about memes below\n1: Choose meme\n2: Save meme\n3: Go back to filters and clear the canvas\n")
                meme_code = int(input("Enter the option from menu: "))
                
                if meme_code==1 :
                   
                    print("\nMenu for items to create meme below\n1: Glasses\n2: Eye Mask\n3: Gas Mask\n4: Face Mask\n5: Black Hat\n6: Snow Hat\n")
                    meme_asset_code = int(input("Enter item for meme: "))
                    
                    if meme_asset_code==1 :
                        
                        meme=eyes_detection(meme,"meme_assets/glasses.png")
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected eyes are smaller than 2\n")
                        
                    elif meme_asset_code==2 :

                        meme = eyes_detection(meme,"meme_assets/eye_mask.png")
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected eyes are smaller than 2\n")
                    elif meme_asset_code==3 :
                        
                        meme = face_detection(meme,"meme_assets/gas_mask.png",False)
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected faces are smaller than 1\n")
                            
                    elif meme_asset_code==4 :

                        meme = face_detection(meme,"meme_assets/face_mask.png",False)
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected faces are smaller than 1\n")
                    elif meme_asset_code==5 :
                        meme = face_detection(meme,"meme_assets/black_hat.png",True)
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected faces are smaller than 1 or cannot put the hat\n")
                    elif meme_asset_code==6 :
                        meme = face_detection(meme,"meme_assets/snow_hat.png",True)
                        if meme is not None :
                            image_from_filters=meme.copy()
                        else :
                            print("\nDetected faces are smaller than 1 or cannot put the hat\n")
                        
                elif meme_code==2 :
                    meme_name = input("Enter a name for saved meme: ")
                    
                    index = meme_name.find('.')
                    if index!=-1 :
                        meme_ending = meme_name[index : ]
                        if meme_ending==".jpeg" :
                            cv2.imwrite(meme_name,cv2.cvtColor(meme,cv2.COLOR_BGRA2RGBA))
                            print("\nSaving the image was successful\n")
                        else :
                            print("\nIncorrect file format for saved image\n")
                    else :
                        print("\nIncorrect file format for saved image\n")
                    
                elif meme_code==3 :
                    
                    image_from_filters=image_cv2.copy()
                    break
                    
        elif code_for_filter==2 :
            
            image_from_filters=gaussian_blur_filter(image_from_filters,5)
        
        elif code_for_filter==3 :
            
            image_from_filters=laplacian_filter(image_from_filters)
            
        elif code_for_filter==4 :
            
            print("\nChannels below\n0: RED\n1: GREEN\n2: BLUE\n")
            channel = int(input("Give channel that you want to make color boosting: "))
            image_from_filters_help=color_boosting(image_from_filters, channel)
            if image_from_filters_help is None :
                print("\nColor Boosting failed!\n")
            else :
                image_from_filters=image_from_filters_help.copy()
                
        elif code_for_filter==5 :
            
            image_from_filters=sharpening_filter(image_from_filters)
            
        elif code_for_filter==6 :
            
            image_from_filters=sepia_filter(image_from_filters)
            
        elif code_for_filter==7 :
            
            image_from_filters=box_blur_filter(image_from_filters)

Enter the image name from folder project_images or Press 'Exit' to exit: Exit
